# Part 3: Pandas

[pandas](https://pandas.pydata.org/) is the primary data analysis library for Python, with DataFrames similar to those used in R.

First import it and set the options to display all columns in the notebook.

In [ ]:
import pandas as pd

pd.options.display.max_columns = None

## Pandas basics
### Series

A series is basically an array with an index of labels. If an index isn't specified a default will be created.

In [ ]:
import random

random.seed(2024)

pd.Series(random.sample(range(1000), 6))

A series _can_ contain multiple types but it's really best if it doesn't.

Using dates for indices can be useful for time series calculations.

In [ ]:
n = 100
ts = pd.Series(
    random.sample(range(1000), n), 
    index = pd.date_range("2024-01-01", periods=n, freq="d"), 
)
ts

We can use slice notation on the index labels to get the data for the first week in February.

In [ ]:
ts.loc["2024-02-01":"2024-02-07"]

Or we can use integer based positioning. Notice that the position based slicing does not include the final value but the label based slice does.

In [ ]:
ts.iloc[0:7]

We can use a boolean mask with `.loc`. For example, to find the days when more than 900 things happened

In [ ]:
ts.loc[ts > 900]

We can assign new values using a `.loc` slice.

In [ ]:
ts.loc[ts > 900] = 10
ts.loc["2024-02-01":"2024-02-07"]

We can resample to get the monthly sum of whatever it is we're counting. 

In [ ]:
ts.resample("ME").sum()

We can use other aggregate functions too.

#### Task 

Find the weekly mean. **Hint** Check the frequency strings [in this section](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

Find the weeks it is above 600.

<details><summary><b>Solution</b></summary>
  
<pre><code>
m = ts.resample("W").mean()
m[m > 600]
 </code></pre>
</details>
<br>

We can use the normal mathematical operators.

In [ ]:
ts * 2

### Dataframes

A dataframe is, simplifying, a load of Series bundled together with both a row and column index.

In [ ]:
df = pd.DataFrame(
    {
        "A": random.sample(range(1000), n),
        "B": random.sample(range(1000), n),
        "C": random.sample(range(1000), n),
    },
    index = pd.date_range("2024-01-01", periods=n, freq="d")
)
df

`.iloc` and `.loc` work in the same way but with an extra dimension.

In [ ]:
df.iloc[:10, 1:]

In [ ]:
df.loc["2024-02-01":"2024-02-07", ["A", "C"]]

It's good practice to use `.loc` and `.iloc` notation as much as possible but it's often convenient to just use square brackets to select one or more columns.

In [ ]:
df["A"] = 1
df

Define a second dataframe with an overlapping index to the first.

In [ ]:
df2 = pd.DataFrame(
    {
        "D": random.sample(range(1000), n),
        "E": random.sample(range(1000), n),
        "F": random.sample(range(1000), n),
    },
    index = pd.date_range("2024-02-01", periods=n, freq="d")
)
df2.resample("ME").sum()

As the indices overlap we can join on the index using `concat` which is far more efficient than using `merge`.

In [ ]:
pd.concat([df.resample("ME").sum(), df2.resample("ME").sum()], axis=1, join="inner")

Interestingly if we use an outer join the values are converted to floats. This is because the original values use `numpy`'s non-nullable int type and would need to be converted to the newer nullable `Int64` first.

In [ ]:
pd.concat([df.resample("ME").sum(), df2.resample("ME").sum()], axis=1, join="outer")

Note that because we're using a common index we don't have to perform a join to make calculations on columns of two dataframes. For example, if we want to add the weekly totals for column `A` in `df` and column `D` in `df2` we can write the following.

In [ ]:
df["A"].resample("W").sum() + df2["D"].resample("W").sum()

### Further reading

Understanding the index and selection operators is key to getting the most out of pandas. The [Getting Started with pandas](https://wesmckinney.com/book/pandas-basics#pandas_summarize) chapter in Wes McKinney's [Python for Data Analysis](https://wesmckinney.com/book/) covers more than we have time for here, and at some point you should read the user guide on [Indexing and selecting data](https://pandas.pydata.org/docs/user_guide/indexing.html) which is very thorough.

It should be noted that many people productively use pandas without touching the index.

## Returning to the worked example
### Data preparation

We would normally use `pd.read_json` or `pd.read_csv` or `pd.read_parquet` etc to read the file but in this case it leaves us with embedded json columns.

In [ ]:
import os

username = os.environ["AWS_ROLE_ARN"].split("/")[-1]
filename = f"s3://alpha-everyone/python_training/{username}/crime.json"
# If you're having access problems uncomment the following
# filename = "crime.json"

pd.read_json(filename)

Instead use `smart_open` and `json` with `pd.json_normalize` to flatten the file.

In [ ]:
import smart_open
import json

with smart_open.open(filename, "r") as f:
    crimes = pd.json_normalize(json.loads(f.read()))
    
crimes

Note that the columns names have been augmented where the json has been unpacked.

In [ ]:
crimes.columns

One way of accessing a dataframe's column is by putting its name in square brackets. The `unique` method can help us determine when a column has no data or would be better as a factor.

Drop the columns without data or we are not interested in.

#### Task

Drop any columns you feel are unnecessary for the analysis.

<details><summary><b>Solution</b></summary>
  
<pre><code>
crimes = crimes.drop(
    columns = [
        "context", "outcome_status", "persistent_id", "id"
    ]
)
 </code></pre>
</details>
<br>

In [ ]:
crimes = crimes.drop(
    columns = [
        # insert your columns here as a list
    ]
)

In [ ]:
crimes.dtypes

We can now start converting the columns to more appropriate types.

In [ ]:
crimes["category"] = crimes["category"].astype("category")

It's easier to assign or reassign multiple columns using the `assign` method. While we're at it rename some of the unpacked columns.

In [ ]:
crimes = crimes.rename(
    columns = {
        "location.latitude": "latitude",
        "location.longitude": "longitude",
        "location.street.name": "street_name",
        "outcome_status.category": "outcome_category",
        "outcome_status.date": "outcome_date"
    }
)

crimes = crimes.assign(
    location_type = crimes["location_type"].astype("category"),
    location_subtype = crimes["location_subtype"].astype("category"),
    month = pd.to_datetime(crimes["month"]),
    latitude = crimes["latitude"].astype("Float64"),
    longitude = crimes["longitude"].astype("Float64"),
    street_name = crimes["street_name"].astype(pd.StringDtype()),
    outcome_category = crimes["outcome_category"].astype("category"),
    outcome_date = pd.to_datetime(crimes["outcome_date"])
)

### An aside on pydbtools

Now we have edited our table and corrected the types we may want to create a database table on Athena. [pydbtools](https://github.com/moj-analytical-services/pydbtools) is a package developed by the data engineers to help use Athena databases. Keeping as much processing as possible as SQL queries on Athena will be faster and avoid memory problems in Jupyterlab. The [documentation](https://moj-analytical-services.github.io/pydbtools/) contains [examples](https://moj-analytical-services.github.io/pydbtools/examples/mojap_tools_demo/) of how to use the library.

For now create a database if it doesn't exist and write the crimes dataframe to a table.

In [ ]:
import pydbtools as pydb

table_name = f"crimes_{username}"
s3_location = f"s3://alpha-everyone/python_training/db/{username}"

pydb.create_database("python_training")
pydb.dataframe_to_table(
    df = crimes,
    database = "python_training",
    table = table_name,
    location = s3_location
)

This can then be queried as usual.

In [ ]:
pydb.read_sql_query(
    f"""
    select * from python_training.{table_name}
    where category = 'violent-crime'
    """
)

Much of what follows would be better off in SQL but today we're learning pandas so 

### Back to the worked example

Here we are recreating the analysis in Part 2 where we look at the most violent crimes per DLR station.

Remember the `.loc` method is important as it defines the part of the dataframe you're working on. Here we're filtering the rows in the first field with a boolean mask, and selecting the columns in the second field. 

Note that the boolean operators in the mask are the Python bitwise operators, so `&` instead of `and`, `|` instead of `or`, and `~` instead of not. These have different operator precedence to the normal boolean operators so we need to keep the separate parts of the `&` expression in brackets.

The brackets are escaped with a backslash as the `.str.contains` method expects a regular expression.

In [ ]:
(
    crimes
    .loc[
        (crimes["category"] == "violent-crime") & (crimes["street_name"].str.contains("\(dlr\)")), 
        ["street_name", "category"]
    ]
    .groupby("street_name")
    .aggregate("count")
    .rename(columns = {"category": "violent_crimes"})
    .sort_values("violent_crimes", ascending=False)
)

Why is Heron Quays relatively rough? Are more people using the station? Let's have a look at station footfall from TFL's data, the latest of which I could find is from 2022.

In [ ]:
with smart_open.open("https://crowding.data.tfl.gov.uk/Network%20Demand/StationFootfall_2022.csv") as f:
    footfall = pd.read_csv(f)

In [ ]:
footfall

Add together the entries and exits so we can calculate 

In [ ]:
footfall["tap_count"] = footfall["EntryTapCount"] + footfall["ExitTapCount"]
footfall["Station"] = footfall["Station"].astype(pd.StringDtype())
footfall_2022 = footfall.groupby("Station")["tap_count"].sum()
footfall_2022

Now looking at all crimes for 2022 only for any TFL station.

In [ ]:
crimes_2022 = (
    crimes
    .loc[crimes["street_name"].str.contains("\(lu|\(dlr") & (crimes["month"].dt.year == 2022)]
    .groupby("street_name")["category"]
    .count()
)
crimes_2022

We need to do a bit of processing to make sure the station names match TFL's, which only specify DLR when there is an alternative station with the same name, as with Canary Wharf.

In [ ]:
lookup = {
    station: station.replace(" (dlr)", "")
    for station in crimes_2022.index
}
lookup

In [ ]:
lookup["Canary Wharf (dlr)"] = "Canary Wharf DLR"
lookup["Canary Wharf (lu Station)"] = "Canary Wharf"
lookup

In [ ]:
crimes_2022.index = [lookup[i] for i in crimes_2022.index]
crimes_2022

We can now calculate the ratio of crimes to station visits.

In [ ]:
crimes_per_footfall = crimes_2022 / footfall_2022

Convert these to percentages as thankfully they're small.

In [ ]:
pct_crimes_per_footfall = 100 * crimes_per_footfall.sort_values(ascending=False)

In [ ]:
pct_crimes_per_footfall.loc[~pct_crimes_per_footfall.isna()]

This tells me that I should get the Jubilee line to Stratford rather than the DLR.

#### Final task

Find out something interesting! Are people more lairy during office Christmas party season? What could a [map](https://plotly.com/python/maps/) of local crime look like?

### Further reading
* [Python for Data Analysis](https://wesmckinney.com/book/) by Wes McKinney, the original author of pandas
* The [Modern Pandas](https://tomaugspurger.net/posts/modern-1-intro/) series of blog posts

#### Alternative dataframes
pandas is not the only option for data processing.
* [duckdb](https://duckdb.org/docs/api/python/overview.html) uses SQL and can be used in conjunction with pandas. Very useful for inequality joins and the like and [integrates nicely with Jupyter](https://duckdb.org/docs/guides/python/jupyter.html).
* [dask](https://docs.dask.org/en/stable/) is designed for larger-than-memory datasets.
* [polars](https://docs.pola.rs/) is a fast alternative to pandas. 

#### Recommended libraries
##### MoJ libraries
* [pydbtools](https://github.com/moj-analytical-services/pydbtools) for accessing Athena databases
* [mojap-metadata](https://github.com/moj-analytical-services/mojap-metadata) for handling metadata
* [arrow-pd-parser](https://github.com/moj-analytical-services/mojap-arrow-pd-parser) for reading and writing files while ensuring metadata conformance
##### Others
* [arrow](https://arrow.apache.org/docs/python/index.html) You may not need to use this directly but almost certainly will behind the scenes. It provides efficient data formats for cross-platform development, including `.parquet` files which are incredibly useful for preserving types and working across R and Python. Allows for chunkable datasets which don't have to be read into memory at once.
* [awswrangler](https://aws-sdk-pandas.readthedocs.io/en/stable/index.html) is useful for dealing with AWS.
* [numpy](https://numpy.org/) is compatible with pandas and has a huge amount of useful functionality for numerical computing.
* [matplotlib](https://matplotlib.org/) is a powerful but complex data visualisation library.
* [seaborn](https://seaborn.pydata.org/) is an easier to use data visualisation library built on matplotlib.
* [plotly](https://plotly.com/python/) is an alternative data visualisation library, particularly useful for...
* [dash](https://dash.plotly.com/) which can be used for webapps.
* [streamlit](https://streamlit.io/) is a simpler webapp framework, useful for single page dashboards.
* [scikit-learn](https://scikit-learn.org/stable/) The starting point for machine learning in Python.